In [ ]:
from selenium import webdriver

In [ ]:
driver = webdriver.Chrome('C:\ProgramFiles\Chrome\chromedriver.exe')
driver.get('https://provtfsapphq-04.tm.nl.ad.tmo/TMNLCol/TMNL/Eagles/_backlogs/TaskBoard/PI%2046/Rapid%2021.14/Sprint%20179?_a=requirements')
sprints = driver.find_elements_by_name('sprints')
c = 0
k = 0
for sprint in sprints:
  sprint.click()
  storys = driver.find_elements_by_name('story')
  for story in storys:
    actionChains = ActionChains(driver)
    action.double_click(story)
    description = driver.find_elements_by_name('discription')
    story_point = driver.find_elements_by_name('story_points')
    with open('s_' + str(c) + '_' + str(k), 'w') as f:
      f.write(description)
    with open('p_' + str(c) + '_' + str(k), 'w') as f:
      f.write(story_point)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sklearn as sk
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import keras_tuner as kt
import re
import nltk
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from gensim.models import Word2Vec
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
!mkdir data

In [ ]:
!unzip drive/MyDrive/arh.zip -d data

# Подключим TPU

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
strategy = tf.distribute.TPUStrategy(resolver)

# Предобработаем данные

In [ ]:
stop_words = stopwords.words('english')
snowball = SnowballStemmer(language='english')

def transform(input_strings):
  output_strings=[]
  for s in input_strings:
    s = re.sub('[^A-Za-z0-9]+', ' ', s).lower().split()
    s = [i for i in s if i not in stop_words]
    s = [snowball.stem(i) for i in s]
    s = ' '.join(s)
    output_strings.append(s)
  return np.array(output_strings)

In [ ]:
x = []
y = []
for i in range(49):
  j = 0
  while True:
    try:
      f = open('data/s_' + str(i) + '_' + str(j) + '.txt', 'r', errors='ignore')
      s = f.read()
      x.append(s)
      f.close()
      f = open('data/p_' + str(i) + '_' + str(j) + '.txt', 'r', errors='ignore')
      y.append([f.read()])
      f.close()
      j += 1
    except:
      break

# Пример данных

In [ ]:
x

In [ ]:
x = np.array(x)
y = np.array(y)

In [ ]:
x = transform(x)

In [ ]:
y = y[x != '']
x = x[x != '']

In [ ]:
e = OneHotEncoder()
data_y = e.fit_transform(y).toarray()

In [ ]:
vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=5000, output_mode='int', output_sequence_length=926)

In [ ]:
vectorize_layer.adapt(x)

In [ ]:
x = vectorize_layer(x).numpy()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, data_y, test_size=0.15, random_state=42)

In [ ]:
N = 1

In [ ]:
def get_model():
  with strategy.scope():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(5000, 200))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200)))
    model.add(tf.keras.layers.Dropout(0.05))
    model.add(tf.keras.layers.Dense(500, activation=tf.nn.tanh))
    model.add(tf.keras.layers.Dropout(0.05))
    model.add(tf.keras.layers.Dense(500, activation=tf.nn.tanh))
    model.add(tf.keras.layers.Dropout(0.05))
    model.add(tf.keras.layers.Dense(500, activation=tf.nn.tanh))
    model.add(tf.keras.layers.Dropout(0.05))
    model.add(tf.keras.layers.Dense(500, activation=tf.nn.tanh))
    model.add(tf.keras.layers.Dropout(0.05))
    model.add(tf.keras.layers.Dense(100, activation=tf.nn.tanh))

    model.add(tf.keras.layers.Dense(7, activation=tf.nn.softmax))
    model.compile(optimizer='Adam', loss='MSLE', metrics=['accuracy'])
    return model

In [ ]:
model = get_model()

In [ ]:
model.fit(x_train, y_train, epochs=5)

In [ ]:
def equals(y_pred, y_real):
  if np.argmax(y_pred) - np.argmax(y_real) in [-1, 0, 1]:
    return 1
  else:
    return 0

In [ ]:
def get_predict():
  y_predict = model.predict(x_test)
  y_predict = e.inverse_transform(y_predict)
  y_predict = e.transform(y_predict).toarray()
  a = np.array([equals(y_predict[i], y_test[i]) for i in range(len(y_test))])
  a = a.sum()
  a = a / len(y_predict)
  return a

In [ ]:
get_predict()